# Datenaufbereitung RCP2.6 Temperatur

###  Packete laden, Pfade definieren und Daten laden

In [1]:
# Benötigte Packete
import xarray as xr
import geopandas as gpd
import rioxarray
import numpy as np
import pandas as pd
from pyproj import Geod
from shapely.geometry import Point
import os
from glob import glob

# Dateipfade definieren
nc_dir = "https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/rcp26temp/raw/"

#Shapefile Pfad
shp_path= "https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/Murgtal_boundingbox/Murgtal_boundingbox.shp"

#Hier muss der lokale Pfad definiert werden:
output_csv = "hier-lokalen-Pfad-definieren/1_daten/rcp26pr/murg_tas_rcp26_mean.csv"

# Laden der Shapefile für das Murgeinzugsgebiet, Shapefile in Koordinatensystem EPSG 4326 umwandeln
murg_shape = gpd.read_file(shp_path).to_crs("EPSG:4326")

#Liste erstellen für Tageswerte
all_data = []

# ALle Dateien holen sortiert nach Jahr
nc_files = sorted(glob(os.path.join(nc_dir, "*.nc")))

### Loop zur Datenaufbereitung aller .nc Daten
Eine Shapefile kann nicht direkt auf eine .nc Datei angewendet werden. Shapefile = Vektoren, NC-Datei = Gitterzellen --> Maske schaut welche Gitterpunkte betroffen sind, sie hat das gleiche Format wie die Klimadaten in der .nc Datei

In [3]:
for nc_path in nc_files:
    print("Verarbeite:", nc_path)
    
    # Datei öffnen
    tas_ds = xr.open_dataset(nc_path)
    tas = tas_ds["tas"]

    # 2D lat/lon extrahieren
    lats = tas_ds["lat"].values
    lons = tas_ds["lon"].values

    # Maske erstellen 
    mask = np.zeros_like(lats, dtype=bool)
    for geom in murg_shape.geometry:
        mask |= np.vectorize(lambda x, y: geom.contains(Point(x, y)))(lons, lats)
    mask_da = xr.DataArray(mask, dims=("rlat", "rlon"))

    # Auf Murg beschränken
    tas_murg = tas.where(mask_da)

    # Mittelwert errechnen
    mean_temp_celsius = (tas_murg.mean(dim=["rlat", "rlon"]) - 273.15).rename("temp_day")

    # Zu DataFrame umwandeln
    df = mean_temp_celsius.to_dataframe().reset_index()
    df = df[["time", "temp_day"]].rename(columns={"time": "date"})

    all_data.append(df)

# Alle Daten zusammenführen
final_df = pd.concat(all_data, ignore_index=True)
final_df["temp_day"] = pd.to_numeric(final_df["temp_day"], errors="coerce")

# CSV speichern
final_df.to_csv(output_csv, index=False)
print("CSV gespeichert:", output_csv)

Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20060101-20101231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20110101-20151231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20160101-20201231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20210101-20251231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20260101-20301231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20310101-20351231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20360101-20401231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20410101-20451231.nc
Verarbeite: data/rcp26temp/tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r5_day_20460101-20501231.nc
Verarbeite

### Erstellen einer CSV mit allen Niederschlagsdaten von 2006 bis 2100

In [27]:
final_df = pd.concat(all_data, ignore_index=True)
final_df.to_csv(output_csv, index=False)
print("Erste Zeilen aus CSV:")
print(pd.read_csv(output_csv).head())

Erste Zeilen aus CSV:
                 datum  temperatur_tag
0  2006-01-01 12:00:00        1.483337
1  2006-01-02 12:00:00        3.466675
2  2006-01-03 12:00:00        4.116699
3  2006-01-04 12:00:00        8.150024
4  2006-01-05 12:00:00        3.916687


### Überblick über den neuen Datensatz

In [32]:
print(df.head())

print(df["temp_day"].describe())

                datum  temperatur_tag
0 2096-01-01 12:00:00        2.883362
1 2096-01-02 12:00:00        1.416656
2 2096-01-03 12:00:00       -0.183319
3 2096-01-04 12:00:00       -2.383331
4 2096-01-05 12:00:00       -2.133331
count    1826.000000
mean       11.918434
std         7.509057
min        -8.916656
25%         5.924995
50%        11.966675
75%        18.533325
max        26.899994
Name: temperatur_tag, dtype: float64
